# Ralphy Núñez Mercado

## T2.3: Scrapear personaxes e capítulos de Rick e Morty

Scraping nivel 0. Servizos web
Apuntes relacionados: https://github.com/jfsanchez/SBD/blob/main/notebooks/scraping/scraping-0-API-REST.ipynb

Exercicio: Das URL seguintes podes obter os episodios e personaxes.

Capítulos: https://rickandmortyapi.com/api/episode
Personaxes: https://rickandmortyapi.com/api/character
Queremos saber por unha banda as personaxes e por outra os capítulos.

Coidado! A API ten un paxinador:

https://rickandmortyapi.com/api/character?page=42
https://rickandmortyapi.com/api/episode?page=3
Tes que recorrer as páxinas ata que en info, next sexa null.

O código debe ser xenérico e ser chamado cunha función tanto para personaxes como para episodios.

Deben gardarse os datos nunha base de datos de PostgreSQL.

### ⬇️📚 Imports necesarios

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

## 🔗 Creando a cadea de conexión

In [2]:
#Datos de conexión
db_host = "10.133.28.46" # ip cesga
db_port=5432 
db_user = "postgres"
db_passwd="Cl431Ns3gur4"
db_name="rickymorty"

## 🔁 Recorrer la API

In [3]:
URL_episodio = "https://rickandmortyapi.com/api/episode"
URL_personaje = "https://rickandmortyapi.com/api/character" 

def recorrer(URL):
    paginas = requests.get(URL).json()["info"]["pages"] 
    resultados = []

    for i in range(1, paginas + 1):
        response = requests.get(f"{URL}?page={i}").json()
        resultados.extend(response["results"])
    
    return resultados  

episodios = recorrer(URL_episodio)
print(f"filas : {len(episodios)}")


filas : 51


(en los episodio guardar  los personajes pero en los personajes no guardar los episodios)

## Creacion de la base de datos postgresql rick y morty

### Primero crearemos el Docker   




```bash
docker run 
\ --name de-postre-sql 
\ -v /home/cesgaxuser/BBDD:/BBDDdocker 
\ -e POSTGRES_PASSWORD=Cl431Ns3gur4 
\ --restart unless-stopped 
\ -p 5432:5432 
\ -p 5433:5433 
\ -d postgres
```


### Entramos en el docker y creamos la base de datos  

```bash
docker exec -it de-postre-sql psql -U postgres
```

### Creación de las tablas para nuestra BD

Episodio:

```sql
CREATE TABLE episodio (id INT, 
    name VARCHAR(255), 
    air_date VARCHAR(255), 
    episode VARCHAR(255), 
    characters text[], 
    url text, 
    created text);
```
Personaje: 

```sql
CREATE TABLE personaje (id INT,   
    name VARCHAR(255), 
    status VARCHAR(255), 
    species VARCHAR(255), 
    type VARCHAR(255), 
    gender VARCHAR(255), 
    origin TEXT, 
    location TEXT, 
    image text, 
    url text, 
    created text);
```

### Ahora tendremos que insertar los datos de la API a las tablas de nuestra base de datos de PostgreSQL ( OJO que en la tabla personaje no vamos a guardar los episodios )

### 🔗 Primero vamos a probar la conexion con la base de datos en postgreSQL

Anteriormente usabamos una conection string como esta:
```python 
    connectionString=f'mysql+pymysql://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'
```
El problema es que esta connectionString es para mysql y en este ejercicio usamos postgreSQL.  

Para poder conectarnos a una BD en postgreSQL tendremos que instalar esta librería: 

In [4]:
# %pip install psycopg2 
# %conda install anaconda::psycopg2

Con la libreria instalada esta sería la connectionString adecuada para conectarnos.

In [5]:
connectionString = f'postgresql+psycopg2://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'

Creamos el engine

In [6]:
from sqlalchemy import create_engine, text
from sqlalchemy import insert

engine = create_engine(connectionString)

Creacion de las tablas desde python

In [ ]:
create_table_episodioSQL = "CREATE TABLE episodio (id INT, name VARCHAR(255), air_date VARCHAR(255), episode VARCHAR(255), characters text[], url text, created text);"
create_table_personajeSQL = "CREATE TABLE personaje (id INT, name VARCHAR(255), status VARCHAR(255), species VARCHAR(255), type VARCHAR(255), gender VARCHAR(255), origin TEXT, location TEXT, image text, url text,created text);"

with engine.connect() as conn:
    conn.execute(text(create_table_episodioSQL))
    conn.execute(text(create_table_personajeSQL))
    conn.commit()

selectSQL = "SELECT * FROM episodio"
dfPanda = pd.read_sql(selectSQL, engine)
print(dfPanda.head())

Obtener los json 

In [8]:
json_episodios = recorrer(URL_episodio)
json_personajes = recorrer(URL_personaje)

### ⬇️ Ahora ya podremos insertar los datos en las tablas.

In [ ]:
with engine.connect() as conn:
    for episodio in json_episodios:
        id = episodio['id']
        name = episodio['name'].replace("'","")
        air_date = episodio['air_date']
        episode = episodio['episode']
        characters = episodio['characters']
        url = episodio['url']
        created = episodio['created']
        
        sql = f"""
            INSERT INTO episodio ( id, name, air_date, episode, characters, url, created) 
            VALUES ({id}, '{name}', '{air_date}', '{episode}', array{str(characters)}, '{url}', '{created}')
        """

        conn.execute(text(sql))
    conn.commit()


with engine.connect() as conn:
    for personaje in json_personajes:
        id = personaje['id']
        name = personaje['name'].replace("'","")
        status = personaje['status'].replace("'","")
        species = personaje['species'].replace("'","")
        typep = personaje['type'].replace("'","")
        gender = personaje['gender'].replace("'","")
        origin = personaje['origin']['name'].replace("'","")
        location = personaje['location']['name'].replace("'","")
        image = personaje['image']
        url = personaje['url']
        created = personaje['created']

         
        sql = f"""
            INSERT INTO personaje ( id, name, status, species, type, gender, origin, location, image, url,created) 
            VALUES ({id}, '{name}', '{status}', '{species}', {f"'{typep}'" if typep != 'NULL' else 'NULL'}, '{gender}','{location}','{origin}' ,'{image}','{url}','{created}')
        """
        conn.execute(text(sql))
    conn.commit()

### ✅ Comprobamos que los datos se han insertado correctamente

In [ ]:
count = "SELECT count(*) FROM episodio "
dfCount = pd.read_sql(count, engine)
print(dfCount)
dfPanda = pd.read_sql(selectSQL,engine)
dfPanda.head()

   count
0     51


,id,name,air_date,episode,characters,url,created
0,1,Pilot,"December 2, 2013",S01E01,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/1,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/2,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/3,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/4,2017-11-10T12:56:34.129Z
4,5,Meeseeks and Destroy,"January 20, 2014",S01E05,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/5,2017-11-10T12:56:34.236Z


In [ ]:
count = "SELECT count(*) FROM personaje "
dfCount = pd.read_sql(count, engine)
print(dfCount)
select_personaje = "SELECT * FROM personaje"
dfselect_personaje = pd.read_sql(select_personaje,engine)
dfselect_personaje.head()

   count
0    826


,id,name,status,species,type,gender,origin,location,image,url,created
0,1,Rick Sanchez,Alive,Human,,Male,Citadel of Ricks,Earth (C-137),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,Citadel of Ricks,unknown,https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z
3,4,Beth Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z
4,5,Jerry Smith,Alive,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z


## MONGO DB

### ⬇️📚 Instalar librerias para poder conectarnos con MongoDB

Para conectarnos a mongoDB, una vez más deberemos descargar la libreria para poder utilizar su respectiva conectionString

In [ ]:
# %conda install -y pymongo dnspython

^C


Retrieving notices: done
Channels:
 - defaults
 - conda-forge
 - anaconda
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ralphy.nunezmercado\AppData\Local\miniconda3\envs\bigdata

  added / updated specs:
    - dnspython
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dnspython-2.4.2            |  py312haa95532_0         480 KB
    pymongo-4.8.0              |  py312h827c3e9_0         1.3 MB
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following NEW packages will be INSTALLED:

  dnspython          pkgs/main/win-64::dnspython-2.4.2-py312haa95532_0 
  pymongo            pkgs/main/win-64::pymongo-4.8.0-py312h827c3e9_0 



pymongo-4.8.0        | 1.3 MB    |            |   0% 

dnspython-2.4.2      | 480 KB    |            |

### 🔗 Conectarnos a Mongo

In [ ]:
from pymongo import MongoClient

#HOST="(...).mongodb.net"
HOST="cluster0.6ej71.mongodb.net"
PORT=27017
USERNAME="ralphynm1000"
PASSWORD="hola1234"

if HOST == 'localhost':
    if not USERNAME:
        cli_mongo = MongoClient(HOST, PORT)
    else:
        cli_mongo = MongoClient(HOST, PORT, USERNAME, PASSWORD)
else:
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/")




['rickymorty', 'sample_mflix', 'admin', 'local']


### 🔗 Probar la conexión con mongo listando las bases de datos.

In [ ]:
print (cli_mongo.list_database_names())

['rickymorty', 'sample_mflix', 'admin', 'local']


### 🔃 Cargar la base datos y mostar las colecciones que tiene

In [ ]:
db_rickymotry=cli_mongo['rickymorty']
print (db_rickymotry.list_collection_names())

['episodios', 'personajes']


### ⬇️ Insertar datos en la coleccion de episodios

In [ ]:
coleccion_episodios=db_rickymotry['episodios']

coleccion_episodios.insert_many(json_episodios)

InsertManyResult([ObjectId('67d85abcc8d275a6fe6e32fd'), ObjectId('67d85abcc8d275a6fe6e32fe'), ObjectId('67d85abcc8d275a6fe6e32ff'), ObjectId('67d85abcc8d275a6fe6e3300'), ObjectId('67d85abcc8d275a6fe6e3301'), ObjectId('67d85abcc8d275a6fe6e3302'), ObjectId('67d85abcc8d275a6fe6e3303'), ObjectId('67d85abcc8d275a6fe6e3304'), ObjectId('67d85abcc8d275a6fe6e3305'), ObjectId('67d85abcc8d275a6fe6e3306'), ObjectId('67d85abcc8d275a6fe6e3307'), ObjectId('67d85abcc8d275a6fe6e3308'), ObjectId('67d85abcc8d275a6fe6e3309'), ObjectId('67d85abcc8d275a6fe6e330a'), ObjectId('67d85abcc8d275a6fe6e330b'), ObjectId('67d85abcc8d275a6fe6e330c'), ObjectId('67d85abcc8d275a6fe6e330d'), ObjectId('67d85abcc8d275a6fe6e330e'), ObjectId('67d85abcc8d275a6fe6e330f'), ObjectId('67d85abcc8d275a6fe6e3310'), ObjectId('67d85abcc8d275a6fe6e3311'), ObjectId('67d85abcc8d275a6fe6e3312'), ObjectId('67d85abcc8d275a6fe6e3313'), ObjectId('67d85abcc8d275a6fe6e3314'), ObjectId('67d85abcc8d275a6fe6e3315'), ObjectId('67d85abcc8d275a6fe6e33

### ⬇️ Insertar los datos en la coleccion de personajes

In [ ]:
coleccion_personajes=db_rickymotry['personajes']

coleccion_personajes.insert_many(json_personajes)

InsertManyResult([ObjectId('67d85c67c8d275a6fe6e3330'), ObjectId('67d85c67c8d275a6fe6e3331'), ObjectId('67d85c67c8d275a6fe6e3332'), ObjectId('67d85c67c8d275a6fe6e3333'), ObjectId('67d85c67c8d275a6fe6e3334'), ObjectId('67d85c67c8d275a6fe6e3335'), ObjectId('67d85c67c8d275a6fe6e3336'), ObjectId('67d85c67c8d275a6fe6e3337'), ObjectId('67d85c67c8d275a6fe6e3338'), ObjectId('67d85c67c8d275a6fe6e3339'), ObjectId('67d85c67c8d275a6fe6e333a'), ObjectId('67d85c67c8d275a6fe6e333b'), ObjectId('67d85c67c8d275a6fe6e333c'), ObjectId('67d85c67c8d275a6fe6e333d'), ObjectId('67d85c67c8d275a6fe6e333e'), ObjectId('67d85c67c8d275a6fe6e333f'), ObjectId('67d85c67c8d275a6fe6e3340'), ObjectId('67d85c67c8d275a6fe6e3341'), ObjectId('67d85c67c8d275a6fe6e3342'), ObjectId('67d85c67c8d275a6fe6e3343'), ObjectId('67d85c67c8d275a6fe6e3344'), ObjectId('67d85c67c8d275a6fe6e3345'), ObjectId('67d85c67c8d275a6fe6e3346'), ObjectId('67d85c67c8d275a6fe6e3347'), ObjectId('67d85c67c8d275a6fe6e3348'), ObjectId('67d85c67c8d275a6fe6e33

## Redis

### ⬇️📚 Instalar la libreria de redis

In [ ]:
#!conda install -y -c conda-forge redis-py sqlalchemy


Para poder conectarnos a correctamente debemos abrir el puerto 6379

```bash 
ssh cesgaxuser@10.133.28.46 -L 6379:localhost:6379
```

### Conectarnos con Redis

In [ ]:
import redis 


r = redis.Redis(host='localhost', port=6379, password="123quetal123")

r.json().set('episodios', '$', json_episodios)
r.json().get('episodios')


